replicate the steps from Brats: https://cbica.github.io/CaPTk/preprocessing_brats.html

1. unify orientations using FSL
2. Register all modalites to T1w
3. Register T1w to template (https://www.nitrc.org/projects/sri24)
4. Perform skull stripping on T1w